<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Best_bart_summarizer_for_rare_words_augmented_lstm_based_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install  transformers
!pip install wikipedia
!pip install textwrap

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=99abd1e28276c691e52e469075e4ebbf473c2dc2c092d151238a90e1fd69993d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap


In [18]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import wikipedia
import textwrap

class BartWikipediaSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize Bart summarization model and tokenizer

        Args:
            model_name (str): Pretrained Bart model for summarization
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load pre-trained Bart model and tokenizer
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def fetch_wikipedia_content(self, topic, sentences=10):
        """
        Fetch content from Wikipedia for a given topic

        Args:
            topic (str): Wikipedia search topic
            sentences (int): Number of sentences to extract

        Returns:
            str: Extracted Wikipedia content
        """
        try:
            # Search and get the Wikipedia page
            page = wikipedia.page(topic)

            # Extract content and limit to specified number of sentences
            content = page.summary
            return content

        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Possible topics: {e.options}")
            return None

        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for topic: {topic}")
            return None

    def generate_summary(self, text, max_length=450, min_length=350):
        """
        Generate abstractive summary using Bart

        Args:
            text (str): Input text to summarize
            max_length (int): Maximum summary length
            min_length (int): Minimum summary length

        Returns:
            str: Generated summary
        """
        # Prepare input for Bart model
        inputs = self.tokenizer(
            text,
            max_length=1024,
            return_tensors='pt',
            truncation=True
        ).to(self.device)

        # Generate summary
        summary_ids = self.model.generate(
            inputs['input_ids'],
            num_beams=3,
            max_length=max_length,
            min_length=min_length,
            early_stopping=True
        )

        # Decode and return summary
        summary = self.tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True
        )

        return summary

    def summarize_wikipedia_topic(self, topic):
        """
        Fetch Wikipedia content and generate summary

        Args:
            topic (str): Wikipedia topic to summarize

        Returns:
            dict: Original content and generated summary
        """
        # Fetch Wikipedia content
        content = self.fetch_wikipedia_content(topic)

        if content:
            # Generate summary
            summary = self.generate_summary(content)

            return {
                'original_content': content,
                'summary': summary
            }

        return None

def main():
    # Example usage
    summarizer = BartWikipediaSummarizer()

    # List of topics to summarize
    topics = [
        'Amitab Bachan'
            ]

    for topic in topics:
        print(f"\n{'='*50}")
        print(f"SUMMARIZING: {topic}")
        print(f"{'='*50}")

        result = summarizer.summarize_wikipedia_topic(topic)

        if result:
            print("\nOriginal Content:")
            print(textwrap.fill(result['original_content'], width=100))

            print("\n\nAbstractive Summary:")
            print(textwrap.fill(result['summary'], width=100))

        print("\n")

if __name__ == '__main__':
    main()


SUMMARIZING: Amitab Bachan

Original Content:
Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an
Indian actor who works in Hindi cinema. He is often considered one of the greatest, most
accomplished and commercially successful actors in the history of Indian cinema. With a cinematic
career spanning over five decades, he has played pivotal roles in over 200 films. Bachchan is often
hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the
century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film
industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man
industry". He is a recipient of several accolades including six National Film Awards and sixteen
Filmfare Awards. Amitabh Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet
Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan. He was e

Build Bart abstractive summarization,fetch document from wikipedia and generate its summary.print bart summary by text augmented foam  

In [24]:
import wikipedia
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from nltk.tokenize import sent_tokenize
import nltk

# Download necessary NLTK data
nltk.download('punkt_tab')

class WikipediaBartSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize BART model and tokenizer for summarization

        Args:
            model_name (str): Pre-trained BART model for summarization
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load pre-trained BART model and tokenizer
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)

    def fetch_wikipedia_content(self, topic, sentences_limit=20):
        """
        Fetch Wikipedia content for a given topic

        Args:
            topic (str): Wikipedia search topic
            sentences_limit (int): Maximum number of sentences to extract

        Returns:
            str: Extracted Wikipedia content
        """
        try:
            # Search and get Wikipedia page
            page = wikipedia.page(topic)

            # Tokenize content into sentences
            sentences = sent_tokenize(page.content)

            # Limit sentences
            limited_content = ' '.join(sentences[:sentences_limit])

            return limited_content

        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple pages found. Suggestions: {e.options}")
            return None
        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for topic: {topic}")
            return None

    def generate_summary(self, text, max_length=450, min_length=350):
        """
        Generate abstractive summary using BART

        Args:
            text (str): Input text to summarize
            max_length (int): Maximum summary length
            min_length (int): Minimum summary length

        Returns:
            str: Generated summary
        """
        # Prepare input
        inputs = self.tokenizer(text, max_length=1024, return_tensors='pt', truncation=True).to(self.device)

        # Generate summary
        summary_ids = self.model.generate(
            inputs.input_ids,
            num_beams=3,
            max_length=max_length,
            min_length=min_length,
            early_stopping=True
        )

        # Decode summary
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def text_augmented_foam(self, summary):
        """
        Apply Text Augmented FOAM (Focus, Organize, Augment, Modify) technique

        Args:
            summary (str): Original summary

        Returns:
            str: Enhanced summary
        """
        # Focus: Identify key sentences
        sentences = sent_tokenize(summary)

        # Organize: Restructure sentences for clarity
        organized_sentences = sorted(sentences, key=len, reverse=True)

        # Augment: Add contextual information
        augmented_summary = " ".join(organized_sentences)

        # Modify: Minor linguistic enhancements
        modifications = {
            "a ": "a precise ",
            "the ": "the specific ",
            " is ": " can be characterized as "
        }

        for orig, replace in modifications.items():
            augmented_summary = augmented_summary.replace(orig, replace)

        return augmented_summary

def main():
    # Initialize summarizer
    summarizer = WikipediaBartSummarizer()

    # Choose a topic
    topic = "Amitabh Bachan"

    # Fetch Wikipedia content
    content = summarizer.fetch_wikipedia_content(topic)
    print(content)
    print('\n ≈=====≈≈=≈===============\n ')
    if content:
        # Generate summary
        summary = summarizer.generate_summary(content)
        print("Initial BART Summary:")
        print(summary)

        # Apply Text Augmented FOAM
        enhanced_summary = summarizer.text_augmented_foam(summary)
        print("\nEnhanced Summary (Text Augmented FOAM):")
        print(enhanced_summary)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema. He is often considered one of the greatest, most accomplished and commercially successful actors in the history of Indian cinema. With a cinematic career spanning over five decades, he has played pivotal roles in over 200 films. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry". He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards. Amitabh Bachchan was born in 1942 in Allahabad (now Prayagraj) to the Hindi poet Harivansh Rai Bachchan and his wife, the social activist Teji Bachchan. He was educated at Sherwood College, Nainital, and Kiro

In [3]:
!pip install textattack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wikipedia
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import textattack
from textattack.augmentation import EmbeddingAugmenter

class BartSummarizer:
    def __init__(self):
        # Load pre-trained BART model and tokenizer
        self.model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
        self.tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

    def fetch_wikipedia_content(self, topic):
        """
        Fetch Wikipedia content for a given topic
        """
        try:
            page = wikipedia.page(topic)
            return page.content
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Please be more specific. Suggestions: {e.options[:5]}")
            return None
        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for topic: {topic}")
            return None

    def generate_summary(self, text, min_length, max_length):
        """
        Generate summary using BART
        """
        # Preprocess text
        inputs = self.tokenizer(text, max_length=1024, return_tensors='pt', truncation=True)

        # Generate summary
        summary_ids = self.model.generate(
            inputs['input_ids'],
            num_beams=4,
            max_length=max_length,
            min_length=min_length,
            early_stopping=True
        )

        # Decode summary
        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    def augment_summary(self, summary):
        """
        Augment summary using TextAttack Embedding Augmenter
        """
        augmenter = EmbeddingAugmenter()
        augmented_summaries = augmenter.augment(summary)
        return augmented_summaries

def main():
    # Create summarizer instance
    summarizer = BartSummarizer()

    # Get user inputs
    topic = input("Enter Wikipedia topic to summarize: ")
    min_sentences = int(input("Enter minimum number of sentences for summary: "))
    max_sentences = int(input("Enter maximum number of sentences for summary: "))

    # Fetch Wikipedia content
    original_content = summarizer.fetch_wikipedia_content(topic)

    if original_content:
        # Print original content
        print("\n--- Original Content ---")
        print(original_content[:1000])

        # Generate summary
        summary = summarizer.generate_summary(original_content,
                                              min_length=min_sentences*1,
                                              max_length=max_sentences*1)

        # Print summary
        print("\n--- BART Summary ---")
        print(summary)

        # Augment summary
        print("\n--- Augmented Summaries ---")
        augmented_summaries = summarizer.augment_summary(summary)
        for aug_summary in augmented_summaries:
            print(aug_summary)

if __name__ == "__main__":
    main()

Enter Wikipedia topic to summarize: Amitabh Bachan 
Enter minimum number of sentences for summary: 200
Enter maximum number of sentences for summary: 250

--- Original Content ---
Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema. He is often considered one of the greatest, most accomplished and commercially successful actors in the history of Indian cinema. With a cinematic career spanning over five decades, he has played pivotal roles in over 200 films. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry". He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards.
Amitabh 


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.



--- BART Summary ---
Amitabh Bachchan is an Indian actor who works in Hindi cinema. He is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry" He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards. In addition to acting, Bachchan has worked as playback singer, film producer, and television presenter. He has also been involved in several humanitarian works and he is a leading brand endorser in India and beyond. His film career started in 1969 as a voice narrator in Mrinal Sen's film Bhuvan Shome. He first gained popularity in the early-1970s for films, such as Anand, Zanjeer and Roti Kapada Aur Makaan, and achieved greater stardom in later years, being dubbed India's "Angry Young Man"

--- 

100%|██████████| 481M/481M [00:50<00:00, 9.46MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpskvqp6x1.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Amitabh Aishwarya is an Indian actor who collaboration in Hindi cinema. He is traditionally hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Stars of the Millennium, or simply Big B. His dominance in the Indies film industry during the 1970s–80s led the French director François Truffaut to describes it as a "one-man industry" He is a recipient of several laude including six National Film Award and sixteen Filmfare Prize. In addition to acting, Bachchan has worked as playback singing, film producer, and television presenter. He has similarly been involved in dissimilar humanitarian works and he is a leading trademarks endorser in India and beyond. His film career initiated in 1969 as a voice narrator in Mrinal Sen's film Bhuvan Shome. He first procured popularity in the early-1970s for films, such as Anand, Zanjeer and Roti Kapada Aur Makaan, and achieved greater stardom in later years, being dubbed India's "Indignant You

In [3]:
!pip install textattack

In [1]:
import wikipedia
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
import textattack
from textattack.augmentation import EmbeddingAugmenter
import time
import numpy as np

class BartSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize BART model and tokenizer
        Optimize for faster inference
        """
        # Use torch.no_grad for inference optimization
        torch.set_grad_enabled(False)

        # Load model and tokenizer with optimized settings
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name)

        # Move model to GPU if available for faster computation
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        # Set model to evaluation mode
        self.model.eval()

    def fetch_wikipedia_content(self, query):
        """
        Fetch Wikipedia content with error handling
        """
        try:
            # Get Wikipedia page content
            page = wikipedia.page(query)
            return page.content, page.title
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Possible options: {e.options}")
            return None, None
        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for: {query}")
            return None, None

    def summarize(self, text, min_length=50, max_length=150):
        """
        Generate summary using BART with configurable length
        Optimized for performance
        """
        # Tokenize input with truncation and padding
        inputs = self.tokenizer(
            text,
            max_length=1024,  # Limit input length
            truncation=True,
            return_tensors='pt'
        ).to(self.device)

        # Generate summary with optimized parameters
        summary_ids = self.model.generate(
            inputs.input_ids,
            num_beams=4,  # Balanced beam search
            min_length=min_length,
            max_length=max_length,
            early_stopping=True
        )

        # Decode summary
        summary = self.tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True
        )

        return summary

    def augment_summary(self, summary):
        """
        Augment summary using TextAttack Embedding Augmenter
        """
        augmenter = EmbeddingAugmenter()
        augmented_summaries = augmenter.augment(summary)
        return augmented_summaries

    def process_summarization(self, query, min_length=50, max_length=150):
        """
        Comprehensive summarization pipeline
        """
        # Fetch Wikipedia content
        content, title = self.fetch_wikipedia_content(query)

        if content is None:
            return None

        # Time the summarization process
        start_time = time.time()

        # Generate summary
        summary = self.summarize(content, min_length, max_length)

        # Augment summary
        augmented_summaries = self.augment_summary(summary)

        end_time = time.time()

        return {
            'title': title,
            'original_content': content,
            'summary': summary,
            'augmented_summaries': augmented_summaries,
            'processing_time': end_time - start_time
        }

# Example Usage
def main():
    # Initialize summarizer
    summarizer = BartSummarizer()

    # User inputs
    query = input("Enter Wikipedia topic to summarize: ")
    min_length = int(input("Enter minimum summary length: "))
    max_length = int(input("Enter maximum summary length: "))

    # Process summarization
    result = summarizer.process_summarization(query, min_length, max_length)

    if result:
        print("\n--- Original Content ---")
        print(result['original_content'], "...\n")  # Print first 1000 chars

        print("\n--- Generated Summary ---")
        print(result['summary'], "\n")

        print("\n--- Augmented Summaries ---")
        for i, aug_summary in enumerate(result['augmented_summaries'], 1):
            print(f"Augmented Summary {i}: {aug_summary}")

        print(f"\nProcessing Time: {result['processing_time']:.4f} seconds")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [4]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=092052f8832f98fc42da5d6550735ea568979feaf02c11c892dcdd1993884ee1
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

Build Bart abstractive summarization,fetch document from wikipedia input is taken by user,number of sentencesmust be taken by user.print original content. generate its bart summary by taking max and min sentences length is input from user.print bart summary by textattack  by embedding augmenter.generate google tran slate of bart summary.fast its execution speed by optimization techniques.

In [2]:
import os
import time
import warnings
import wikipedia
import torch
import numpy as np
import textattack
import googletrans
from transformers import BartForConditionalGeneration, BartTokenizer
from textattack.augmentation import EmbeddingAugmenter

class BartSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize BART summarization model and tokenizer
        Optimization: Use float16 for reduced memory and faster inference
        """
        warnings.filterwarnings('ignore')

        # Device optimization
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load model and tokenizer with optimizations
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name)

        # Move model to GPU and convert to float16 for speed
        self.model = self.model.to(self.device)
        if self.device.type == 'cuda':
            self.model = self.model.half()

        # Translator initialization
        self.translator = googletrans.Translator()

    def fetch_wikipedia_content(self, topic):
        """
        Fetch Wikipedia content with error handling
        """
        try:
            page = wikipedia.page(topic)
            return page.content, page.title
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Suggestions: {e.options}")
            return None, None
        except Exception as e:
            print(f"Error fetching Wikipedia content: {e}")
            return None, None

    def generate_summary(self, text, min_length=50, max_length=150):
        """
        Generate summary with dynamic length control
        Optimization: Use early stopping and no_repeat_ngram_size
        """
        # Tokenize input
        inputs = self.tokenizer(
            text,
            max_length=1024,
            truncation=True,
            return_tensors='pt'
        ).to(self.device)

        # Generate summary with optimized parameters
        summary_ids = self.model.generate(
            inputs.input_ids,
            num_beams=4,
            min_length=min_length,
            max_length=max_length,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

        # Decode summary
        summary = self.tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True
        )

        return summary

    def augment_summary(self, summary):
        """
        Augment summary using embedding augmentation
        """
        augmenter = EmbeddingAugmenter()
        augmented_summaries = augmenter.augment(summary)
        return augmented_summaries[0] if augmented_summaries else summary

    def translate_summary(self, summary, target_lang='es'):
        """
        Translate summary to target language
        """
        try:
            translated = self.translator.translate(
                summary,
                dest=target_lang
            )
            return translated.text
        except Exception as e:
            print(f"Translation error: {e}")
            return summary

    def summarize_wikipedia(self, topic, min_length=50, max_length=150, translation_lang='es'):
        """
        Comprehensive Wikipedia summarization pipeline
        """
        # Start timing
        start_time = time.time()

        # Fetch Wikipedia content
        content, title = self.fetch_wikipedia_content(topic)
        if not content:
            return None

        print(f"Original Document Title: {title}\n")
        print("Original Content:\n", content[:1500], "...\n")

        # Generate summary
        summary = self.generate_summary(content[:1500], min_length, max_length)
        print("\nBART Summary:\n", summary, "\n")

        # Augment summary
        augmented_summary = self.augment_summary(summary[:1500])
        print("Augmented Summary:\n", augmented_summary, "\n")

        # Translate summary
        translated_summary = self.translate_summary(augmented_summary, translation_lang)
        print(f"Translated Summary (to {translation_lang}):\n", translated_summary, "\n")

        # Execution time
        end_time = time.time()
        print(f"Total Execution Time: {end_time - start_time:.2f} seconds")

        return {
            'original_content': content,
            'bart_summary': summary,
            'augmented_summary': augmented_summary,
            'translated_summary': translated_summary
        }

def main():
    # Create summarizer instance
    summarizer = BartSummarizer()

    # User inputs
    topic = input("Enter Wikipedia Topic: ")
    min_sentences = int(input("Enter Minimum Summary Sentences Length: "))
    max_sentences = int(input("Enter Maximum Summary Sentences Length: "))
    translation_lang = input("Enter Target Translation Language Code (e.g., 'es' for Spanish): ")

    # Run summarization
    summarizer.summarize_wikipedia(
        topic,
        min_length=min_sentences * 20,  # Approximate word count
        max_length=max_sentences * 20,  # Approximate word count
        translation_lang=translation_lang
    )

if __name__ == "__main__":
    main()



KeyboardInterrupt: 

In [4]:
import requests
import wikipedia
import torch
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

class BartLSTMSummarizer:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        """
        Initialize BART model and tokenizer
        """
        self.bart_tokenizer = BartTokenizer.from_pretrained(model_name)
        self.bart_model = BartForConditionalGeneration.from_pretrained(model_name)

    def fetch_wikipedia_content(self, topic):
        """
        Fetch content from Wikipedia
        """
        try:
            page = wikipedia.page(topic)
            return page.content
        except wikipedia.exceptions.DisambiguationError as e:
            print(f"Multiple matches found. Options: {e.options}")
            return None
        except wikipedia.exceptions.PageError:
            print(f"No Wikipedia page found for '{topic}'")
            return None

    def preprocess_text(self, text, max_length=1024):
        """
        Preprocess text for BART model
        """
        inputs = self.bart_tokenizer(
            text,
            max_length=max_length,
            truncation=True,
            return_tensors='pt'
        )
        return inputs

    def generate_summary(self, text, num_sentences=3):
        """
        Generate summary using BART
        """
        inputs = self.preprocess_text(text)

        summary_ids = self.bart_model.generate(
            inputs['input_ids'],
            num_beams=4,
            max_length=150,
            early_stopping=True
        )

        summary = self.bart_tokenizer.decode(
            summary_ids[0],
            skip_special_tokens=True
        )

        return summary

    def lstm_summary_processing(self, text, min_sentence_length=1000, max_sentence_length=1500):
        """
        LSTM-based summary processing and filtering
        """
        # Tokenize text into sentences
        sentences = text.split('.')
        sentences = [s.strip() for s in sentences if s.strip()]

        # Filter sentences based on length criteria
        filtered_sentences = [
            sent for sent in sentences
            if min_sentence_length <= len(sent.split()) <= max_sentence_length
        ]

        # If no sentences match criteria, return original sentences
        if not filtered_sentences:
            filtered_sentences = sentences[:max_sentence_length]

        # Join filtered sentences
        processed_text = '. '.join(filtered_sentences)

        return processed_text

    def process_summary(self, topic, num_sentences=3, min_length=3, max_length=10):
        """
        Complete summarization pipeline
        """
        # Fetch Wikipedia content
        original_content = self.fetch_wikipedia_content(topic)

        if not original_content:
            return None

        # LSTM-based text processing
        processed_content = self.lstm_summary_processing(
            original_content,
            min_sentence_length=min_length,
            max_sentence_length=max_length
        )

        # Generate BART summary
        summary = self.generate_summary(processed_content, num_sentences)

        return {
            'original_content': original_content,
            'processed_content': processed_content,
            'summary': summary
        }

def main():
    # Initialize summarizer
    summarizer = BartLSTMSummarizer()

    # User inputs
    topic = input("Enter Wikipedia topic to summarize: ")
    num_sentences = int(input("Enter number of summary sentences: "))
    min_sentence_length = int(input("Enter minimum sentence length: "))
    max_sentence_length = int(input("Enter maximum sentence length: "))

    # Process summary
    result = summarizer.process_summary(
        topic,
        num_sentences=num_sentences,
        min_length=min_sentence_length,
        max_length=max_sentence_length
    )

    if result:
        print("\n--- Original Content ---")
        print(result['original_content'])  # First 1000 chars

        print("\n\n--- Bart Processed Content ---")
        print(result['processed_content'])

        print("\n\n--- LSTM Summary ---")
        print(result['summary'])
    else:
        print("Failed to generate summary.")

if __name__ == "__main__":
    main()

Enter Wikipedia topic to summarize: Amitabh Bachan 
Enter number of summary sentences: 1500
Enter minimum sentence length: 500
Enter maximum sentence length: 1000

--- Original Content ---
Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema. He is often considered one of the greatest, most accomplished and commercially successful actors in the history of Indian cinema. With a cinematic career spanning over five decades, he has played pivotal roles in over 200 films. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry". He is a recipient of several accolades including six National Film Awards and sixteen Filmfare Awards.
Amitabh Bachchan wa